# NAIVE BAYES ALGORITHM

In [1]:
#OUR LIBRAIRIES
import numpy as np
import pandas as pd
import numpy.linalg as LA
from sklearn.feature_extraction.text import CountVectorizer

## LOAD OUR DATA SET
## here as we have two datasets we have to concatenate its first

In [2]:

data1 = pd.read_table('drugLibTest_raw.tsv',sep='\t')
data2 = pd.read_table('drugLibTrain_raw.tsv',sep='\t')
data = pd.concat((data1,data2), axis=0)
data.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...
1,3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...
2,3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...
3,969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...
4,696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...


then i will group the data according to the length, 
and for that I will use dummies to convert my strings values in
numerical values for the data having a small length and countvectorize for the data having
a long length for the conversion

In [3]:
d1 = data[['effectiveness','sideEffects']]
#data['benefitsReview','commentsReview']= data['benefitsReview','commentsReview'].str.lower()
text= data[['benefitsReview','sideEffectsReview','commentsReview']].astype(str).apply(lambda z: ' '.join(z),axis=1)
text = text.str.lower()
texte= text.replace('[\d+]')#remove numbers
d2 = texte
data3 = pd.get_dummies(d1)
 #list of text documents
text = d2
# create the transform
vectorizer = CountVectorizer( binary = True ,min_df=50,
                            stop_words ='english', max_features =50,
                                                         )
# tokenize and build vocab
tati = vectorizer.fit_transform(text)
data_text=tati.toarray()

 after that i transform it into dataframe and concatenate the both 

In [4]:
data_text = pd.DataFrame(data_text)
D = pd.concat([pd.DataFrame(data3.values),pd.DataFrame(data_text.values)],axis=1)
D

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4138,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4139,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
4140,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,1,0,0,0,0
4141,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Define my features and target. here i use the whole dataset ie 100%

In [5]:
data['rating'] =data['rating'].apply(lambda x  :x-1)
y=data['rating'].values
x = D.values

## Define the function

In [6]:
class NaiveBayes(object):
    
    '''
    The naive Bayes algorithm (multiclassification algorithm) calculate the probability of 
    a piece of data belonging to a given class, given our prior knowledge
        for this we will start by separate the target by class then summarize the dataset, sumarize data by class,
        use the Gaussian Probability Density Function and predict the class probability
  
    '''
    def fit(self,x,y):
        n_samples,n_features = x.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        #initialize, mean, var , prior
        self._mean = np.zeros((n_classes,n_features))
        self._var = np.zeros((n_classes,n_features))
        self._prior = np.zeros((n_classes,))
        for c in self.classes:
            xc = x[c==y]
            self._mean[c,:] = xc.mean(axis=0)
            self._var[c,:] = xc.var(axis=0)
            self._prior[c] = xc.shape[0] / float(n_samples)
                
            
    def predict(self,X):
        y_pred = [self._predict(i) for i in X]
        return y_pred
    
    # Calculate the Gaussian probability distribution function for x
    def pdf(self, class_index, x):
        mean = self._mean[class_index]
        var = self._var[class_index]
        numerator = np.exp(-(x-mean)**2 / (2*var))
        denominator = np.sqrt(2*np.pi*var)
        return numerator/denominator
    
    ## Predict the class for a given row
    def _predict(self, x):
        post = []
        for index,c in enumerate(self.classes):
            prior = np.log(self._prior[index])
            class_conditional = np.sum(np.log(self.pdf(index,x)))
            posterior = prior + class_conditional
            post.append(posterior)
        return self.classes[np.argmax(post)]
            



Spliting my data into training and testing 

In [7]:
X_train = x
X_test = x
Y_train = y
Y_test = y

fitting the model using my training and testing data sets

In [8]:
nb = NaiveBayes()
r =nb.fit(X_train, Y_train)


## The prediction

In [9]:
predictions = nb.predict(X_test)
predictions

/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


[6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,


## The accuracy

In [10]:
def accuracy(y_true,y_pred):
    acc =np.sum(y_true==y_pred) /len(y_true)
    return acc *100


In [11]:
acc = accuracy(Y_test,predictions)
acc

11.368573497465604

## Using 10% of data

In [12]:
percentage = int(len(x)* 10/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x
X_test = x
Y_train = y
Y_test = y

In [13]:
nb = NaiveBayes()
r =nb.fit(X_train, Y_train)
predictions = nb.predict(X_test)
predictions

/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [14]:
def accuracy(y_true,y_pred):
    acc =np.sum(y_true==y_pred) /len(y_true)
    return acc *100
acc = accuracy(Y_test,predictions)
acc

3.3816425120772946

## Using 30% of data

In [15]:
percentage = int(len(x)* 30/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x
X_test = x
Y_train = y
Y_test = y

In [16]:
nb = NaiveBayes()
r =nb.fit(X_train, Y_train)
predictions = nb.predict(X_test)
predictions

/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [17]:
def accuracy(y_true,y_pred):
    acc =np.sum(y_true==y_pred) /len(y_true)
    return acc *100
acc = accuracy(Y_test,predictions)
acc

8.870967741935484

## Using 60% of the data

In [18]:
percentage = int(len(x)* 60/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x
X_test = x
Y_train = y
Y_test = y

In [19]:
nb = NaiveBayes()
r =nb.fit(X_train, Y_train)
predictions = nb.predict(X_test)
predictions

/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [20]:
def accuracy(y_true,y_pred):
    acc =np.sum(y_true==y_pred) /len(y_true)
    return acc *100
acc = accuracy(Y_test,predictions)
acc

9.45945945945946

## Observations

we note that , using all the data we obtain a rather average perfomance of the model. performance decreases when data are reduced. We can conclude that the naive bayes model is not appropriate for this type of data